# Numbers

Scrpt to generate basic numbers such as the number of mice and the number of cells per mouse.

In [46]:
import sys
import pandas as pd
import numpy as np
sys.path.append("/Users/federicoclaudi/Documents/Github/LocomotionControl")

from data.dbase.db_tables import (
    Mouse,
    ValidatedSession,
    Tracking,
    SessionCondition,
    LocomotionBouts,
    Surgery, 
    Probe, 
    Recording, 
    Unit,
)

In [48]:
# get the number of sessions that are implanted vs baseline
sessions = pd.DataFrame(ValidatedSession * SessionCondition)

baseline = sessions[sessions["condition"] == "naive"]
implanted = sessions[sessions["condition"] != "naive"]

print(f"Of a total of {len(sessions)} sessions, {len(baseline)} are baseline and {len(implanted)} are implanted")

# for each group of session, get the number of mice
baseline_mice = len(baseline["mouse_id"].unique())
implanted_mice = len(implanted["mouse_id"].unique())
print(f"Of a total of {len(sessions['mouse_id'].unique())} mice, {baseline_mice} are baseline and {implanted_mice} are implanted")

# get the average and std of the number of sessions per mouse
baseline_mu, baseline_std = np.mean(baseline.groupby("mouse_id").count()["name"].values), np.std(baseline.groupby("mouse_id").count()["name"].values)
implanted_mu, implanted_std = np.mean(implanted.groupby("mouse_id").count()["name"].values), np.std(implanted.groupby("mouse_id").count()["name"].values)

print(f"Baseline mice have {baseline_mu:.2f} +/- {baseline_std:.2f} sessions, while implanted mice have {implanted_mu:.2f} +/- {implanted_std:.2f} sessions")


Of a total of 48 sessions, 7 are baseline and 41 are implanted
Of a total of 7 mice, 1 are baseline and 6 are implanted
Baseline mice have 7.00 +/- 0.00 sessions, while implanted mice have 6.83 +/- 4.95 sessions


In [50]:
# Get the number of trials (locomotion bouts) per session
# TODO get and print the number of trials per session in each direction,  differentiating baseline and implanted
LocomotionBouts()

mouse_id,name,start_frame,end_frame last frame of locomotion bout,duration duration in seconds,direction 'outbound' or 'inbound' or 'none',color,complete True if its form reward to trigger ROIs,start_roi,end_roi,gcoord_delta the change in global coordinates during the bout


### units

In [93]:
mos_regions = ["MOs1", "MOs2/3", "MOs5", "MOs6a"]
mrn_regions = ["PPN", "CUN"]
rsp_regions = ['RSPagl2/3', 'RSPagl5', 'RSPagl6a', 'RSPagl6b', 'RSPd1', 'RSPd2/3', 'RSPd5', 'RSPd6a']
vis_regions = ['VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a']
orb_regions = ['ORBl5', 'ORBl6a', 'ORBvl2/3', 'ORBvl5', 'ORBvl6a']

brain_regions = dict(
    mos=mos_regions,
    mrn=mrn_regions,
    rsp=rsp_regions,
    vis=vis_regions,
    orb=orb_regions,
)

In [99]:
# Get the number of recordings per implanted mice.

mos_mice = (Mouse * Surgery & "target='MOs'")
mrn_mice = (Mouse * Surgery & "target='CUN/GRN'")

print(f"A total of {len(mos_mice)} mice were implanted in MOs, {len(mrn_mice)} were implanted in MRN")

# Get the number of recordings per session.
mos_recs = pd.DataFrame((Recording * Surgery & "target='MOs'").fetch())
mrn_recs = pd.DataFrame((Recording * Surgery & "target='CUN/GRN'").fetch())

print(f"A total of {len(mos_recs)} recordings were made in MOs, {len(mrn_recs)} recordings were made in MRN")

# get the average and std of the number of recordings per session
mos_mu, mos_std = mos_recs.groupby("mouse_id").count().name.mean(), mos_recs.groupby("mouse_id").count().name.std()
mrn_mu, mrn_std = mrn_recs.groupby("mouse_id").count().name.mean(), mrn_recs.groupby("mouse_id").count().name.std()

print(f"{mos_mu:.2f} +/- {mos_std:.2f} recordings per mouse for MOs, {mrn_mu:.2f} +/- {mrn_std:.2f} recordings per mouse for MRN")


# Get the number of units per brain region
units = pd.DataFrame((Unit * Probe.RecordingSite ).fetch())
print("\n\n")
for brain_region, regions in brain_regions.items():
    print(f"Number of units in {brain_region}: {len(units[units['brain_region'].isin(regions)])}")
    for subregion in regions:
        n = len(units[units['brain_region'] == subregion])
        if n == 0:
            continue
        print(f"\t{subregion}: {n}")

A total of 2 mice were implanted in MOs, 6 were implanted in MRN
A total of 12 recordings were made in MOs, 20 recordings were made in MRN
12.00 +/- nan recordings per mouse for MOs, 4.00 +/- 2.45 recordings per mouse for MRN



Number of units in mos: 475
	MOs1: 97
	MOs2/3: 99
	MOs5: 169
	MOs6a: 110
Number of units in mrn: 185
	PPN: 75
	CUN: 110
Number of units in rsp: 16
	RSPagl2/3: 7
	RSPagl6b: 4
	RSPd1: 3
	RSPd2/3: 2
Number of units in vis: 23
	VISp1: 18
	VISp6a: 5
Number of units in orb: 73
	ORBl5: 4
	ORBl6a: 36
	ORBvl5: 2
	ORBvl6a: 31
